# Playlist analyzer

Because of what happened with my account being flagged, this current code doesn't work. It was working perfectly until class, but I will have to find a workaround that doesn't use the api to find song data on the user's playlist. 

## Import the spotify api

In [8]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

# Define your credentials and redirection URI
SPOTIPY_CLIENT_ID = 'dda606479dd244a9b0948c01f155ac7a'
SPOTIPY_CLIENT_SECRET = 'd714739739a04b0fa662e521e1a85ff8'
SPOTIPY_REDIRECT_URI = 'http://localhost:3000'

# Create the Spotify client

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET,
    redirect_uri=SPOTIPY_REDIRECT_URI,
))

user_name = input("Enter your spotify ID: ")
playlists = spotify.user_playlists(user_name)
name = input("Enter playlist name: ")


# https://stackoverflow.com/questions/39086287/spotipy-how-to-read-more-than-100-tracks-from-a-playlist
def get_playlist_tracks(playlist_id):
    results = spotify.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = spotify.next(results)
        tracks.extend(results['items'])
    return tracks

playlist_id = 0
for i in playlists["items"] : 
    if i["name"] == name :
        playlist_id = i["id"]
playlist_data = get_playlist_tracks(playlist_id)


## Extract song data on each song

In [9]:
playlist = []
names = []

for i in playlist_data : 
    names.append(i["track"]["name"])
    playlist.append(spotify.audio_features(i["track"]["id"])[0])

print(playlist[1])


{'danceability': 0.492, 'energy': 0.762, 'key': 8, 'loudness': -6.813, 'mode': 1, 'speechiness': 0.0388, 'acousticness': 0.000551, 'instrumentalness': 0.125, 'liveness': 0.366, 'valence': 0.413, 'tempo': 129.682, 'type': 'audio_features', 'id': '3tYxhPqkioZEV5el3DJxLQ', 'uri': 'spotify:track:3tYxhPqkioZEV5el3DJxLQ', 'track_href': 'https://api.spotify.com/v1/tracks/3tYxhPqkioZEV5el3DJxLQ', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3tYxhPqkioZEV5el3DJxLQ', 'duration_ms': 249515, 'time_signature': 4}


### Convert the new data into a dataframe including the name of each song

In [22]:
collumns = ["danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "type", "id", "uri", "track_href", "analysis_url", "duration_ms", "time_signature"]
selected_columns = ["energy", "acousticness", "valence", "speechiness"]

df = pd.DataFrame(playlist, columns=collumns)
df = df[selected_columns]
df["name"] = names

# scaler = MinMaxScaler()
# scaler = scaler.fit(df[["tempo"]])

# df["tempo"] = pd.DataFrame(scaler.transform(df[["tempo"]]))
df.shape





(52, 5)

## Find the distance of each song to the central point of the playlist's cluster

In [11]:
model = KMeans(n_clusters=1)
model.fit(df[selected_columns])


centroid = model.cluster_centers_[0]
distances = []

for i in range(len(df)) :
    distance = cdist([df[selected_columns].iloc[i]], [centroid], metric="euclidean")
    distances.append(distance[0][0])

df["distance"] = distances

Q1 = df["distance"].quantile(0.25)
Q3 = df['distance'].quantile(0.75)

max = Q3 + (Q3-Q1)*2
print(max)

df.loc[df["distance"] > max]



0.5884934531543962


,energy,acousticness,valence,speechiness,name,distance
13,0.476,0.397,0.234,0.0249,Dream On,0.588674


## Load in the data

In [55]:
df = pd.read_csv("data.csv")
df['energy'] = pd.to_numeric(df['energy'], errors='coerce')


/var/folders/k6/qgdgj9qj743ff3dn87fyv0yr0000gn/T/ipykernel_2575/3618754505.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data.csv")


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982929 entries, 0 to 1982928
Data columns (total 5 columns):
 #   Column        Dtype  
---  ------        -----  
 0   energy        float64
 1   acousticness  float64
 2   valence       float64
 3   speechiness   float64
 4   title         object 
dtypes: float64(4), object(1)
memory usage: 75.6+ MB


## Find recommendations

In [58]:
recommendations = []
for i in range(len(df)) :
    distance = cdist([df[selected_columns].iloc[i].values], [centroid], metric="euclidean")
    if (distance < 0.5) :
        recommendations.append(df.iloc[i]["title"])
    if len(recommendations) >= 50 :
        break

print(recommendations)



['Psycho', 'Amazing World (2011)', 'Turn Back My Love (2009)', 'Time Flies So Fast (2008)', 'The Steel (2010)', 'The Final Edge (2009)', 'The Evilsign (2007)', 'Song of Extasy (2007)', 'Self Control (2008)', 'Road Love (2006)', 'Right Now (Strong Kick) 2009', 'Return to Space Dizzy (2009)', 'Protection (2011)', 'Promise of Love (2011)', 'Played Alive (2011)', 'Paralyzed Synth (2010)', 'Over of Time (2007)', 'Noise Love (2008)', 'New Vision (2010)', 'New Prophercy V (2006)', 'New Prophercy III (2006)', 'New Prophercy (2006)', 'Need for Speed (2009)', 'Mystic of Angels (2011)', 'Mystic Love (2009)', 'Mystic Flower (2008)', 'Mind Wave (2008)', 'Mild More (2008)', 'Love Kiss (2007)', 'Love in My Memory (2010)', 'Last Week (2007)', 'Just Right Now (2007)', 'Join Us (2010)', 'Invitation (2009)', 'Invisible (2008)', "I'm So Alone (2009)", 'I Have a Dream (2010)', 'I Fleer (2008)', 'I Cannot Find You (2010)', 'Help Me Escape (2009)', 'Harmonic Day (2010)', 'Growdraw (2010)', 'Groove Mystic (20

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982929 entries, 0 to 1982928
Data columns (total 5 columns):
 #   Column        Dtype  
---  ------        -----  
 0   energy        object 
 1   acousticness  float64
 2   valence       float64
 3   speechiness   float64
 4   title         object 
dtypes: float64(3), object(2)
memory usage: 75.6+ MB
